# 有望客字詞關聯圖 - Word2Vec

In [24]:
import pandas as pd
import numpy as np
from gensim.models import word2vec
from gensim import models
import ast
from collections import Counter

In [39]:
df = pd.read_csv('./data/sentiment/nissan_clean_data.csv')
df = df[['system_id', 'words']]

In [40]:
df['words'] = df['words'].apply(lambda x: ast.literal_eval(x))
df['words'] = df['words'].apply(lambda x: ' '.join(x))
df

,system_id,words
0,1,新聞 小休 熱鬧 好玩 PEUGEOT300815LBlueH
1,2,情報 2020 11 月份 臺灣汽車 市場 銷售 報告 新增 小七車
2,3,新聞 豐田 PremioAllionPriusAlpha 明年 停產 標題 房車 轎式 MP...
3,4,缺點 選車 重點 new juke 試駕 影片 慢慢 釋出 8891 影片 結尾 依舊 提出
4,5,菜單 NissanAllNEWSentra 尊爵型 Nissan2020All New Se...
...,...,...
2074,2459,購車 休旅車 HondaToyotaLexusNissan SUV 1800cc 以內 20...
2075,2461,售車 Nissan2013BIGTIIDA5 NISSAN 20131 BIG TIIDA ...
2076,2462,售車 Nissankicks 智行 旗艦版 2019 2019 06 1498 顏色 排檔 ...
2077,2463,售車 自售 NISSANTEANAJ31200520 2005 06 J31 TEANA 顏...


In [12]:
df.to_csv('./data/word2vec/nissan_w2v.txt', sep='\t', index=False, header=True, encoding='utf-8')


In [14]:
output = open('./data/word2vec/nissan_w2v.txt', 'w', encoding='utf-8')
for index, row in df.iterrows():
    output.write(row['words'] + '\n')

### 訓練模型

In [16]:
# 讀取已斷詞的資料集
sentences = []
with open('./data/word2vec/nissan_w2v.txt', 'r', encoding='utf-8') as file:
    for line in file:
        # 假設每行是以空白分隔的詞彙
        words = line.strip().split()
        sentences.append(words)
model = word2vec.Word2Vec(sentences, sg=1, window=6, min_count=30, workers=4, vector_size=250)

# 保存模型，供日後使用
model.save("./data/word2vec/nissan_word2vec.model")

### 查看相關詞彙

In [21]:
# 可以使用model.wv.most_similar()來查詢相似詞彙
similar_words = model.wv.most_similar('altis', topn=10)
for word, similarity in similar_words:
    print(word, similarity)

FOCUS 0.8920780420303345
焦點 0.880046010017395
不輸 0.8784715533256531
camry 0.8551652431488037
這代 0.8547437787055969
沒力 0.8538542985916138
佛心 0.8529285192489624
哪台 0.8526982069015503
140 0.8512624502182007
仙草 0.8394905924797058


### 依據資料集的詞頻去找相關字詞

詞頻計算

In [41]:
# 將所有的詞彙合併為一個大字串
all_words = ' '.join(df['words'])

In [44]:
# 使用Counter計算詞頻
word_counts = Counter(all_words.split())

count_df = pd.DataFrame.from_dict(word_counts, orient='index').reset_index()
count_df.columns = ['word', 'freq']
count_df.sort_values(ascending=False, by='freq',inplace=True)
count_df

,word,freq
474,問題,1497
317,空間,1019
312,價格,851
97,業務,812
61,原廠,776
...,...,...
22673,附件,1
22672,低額,1
22670,樂勝馬,1
22668,威根,1


建立字詞correlation

In [48]:
df_most_correlation = pd.DataFrame(columns=['item1', 'item2', 'correlation'])
df_most_correlation

,item1,item2,correlation


定義相關字詞函式

In [46]:
def get_top_similar_word(word, n):
    similar_words = model.wv.most_similar(word, topn=n)
    return similar_words

In [49]:
for index, row in count_df.iterrows():
    try:
        topn = get_top_similar_word(row['word'], 10)
    except:
        print(row['word'])
        continue
    for item in topn:
        tmp = {'item1': row['word'], 'item2': item[0], 'correlation': str(item[1])}
        df_most_correlation = df_most_correlation.append(tmp, ignore_index = True)

AttributeError: 'DataFrame' object has no attribute 'append'

In [5]:
data = df.copy()
data = data[~data.item1.isna()]
data = data[~data.item2.isna()]
data

,item1,item2,correlation
0,nissan,nnissan,0.667420
1,nissan,nismo,0.574859
2,nissan,今日,0.560769
3,nissan,裕隆日產,0.556396
4,nissan,主角,0.541733
...,...,...,...
1995,台灣人,照樣,0.711645
1996,台灣人,好意思,0.709959
1997,台灣人,中共,0.707242
1998,台灣人,看不起,0.702805


In [6]:
data = data[(data['correlation'] <= 0.75) & (data['correlation'] > 0.7)]
data

,item1,item2,correlation
80,業務,業代,0.745061
90,空間,寬敞,0.704260
120,油電,hybrid,0.702189
240,focus,active,0.729345
401,後座,膝部,0.739037
...,...,...,...
1995,台灣人,照樣,0.711645
1996,台灣人,好意思,0.709959
1997,台灣人,中共,0.707242
1998,台灣人,看不起,0.702805


計算 adjency matrix

In [7]:
pers = np.unique(data[['item1', 'item2']])
mat = pd.pivot_table(data,index = 'item1', columns = 'item2' ,values='correlation').fillna(0)\
  .reindex(columns=pers, index=pers, fill_value=0).to_numpy() # 有向圖
print(mat.shape)
tri = (np.tril(mat,-1).T + np.triu(mat,1))
mat_s = tri+tri.T

(291, 291)


繪製網路圖

In [18]:
def random_color():
  r = lambda: random.randint(0,255)
  return '#%02X%02X%02X' % (r(),r(),r())

# input matrix, output graph
def matPresentGraph(mat:np.array,node_id:list,node_value:list = None,edge_color=None):

  if edge_color == None:
    edge_color = random_color()
  edge_color = random_color()
  if node_value == None:
    node_value = [1 for i in range(len(node_id))]

  # cdn_resource : 指定網路圖的CDN資源  
  net = Network(notebook=True, cdn_resources='in_line')

  titles_list = []
  for i in zip(node_id):
    titles_list.append(str(i))

  net.add_nodes(
      nodes = node_id,
      value = node_value,
      label = node_id,
      title = titles_list
  )

  for row in range(len(node_id)):
    for col in range(len(node_id)):
      if mat[row][col]>0.:
        net.add_edge(
            node_id[row],node_id[col],width = mat[row][col],color = edge_color,title = mat[row][col]
        )

  # 計算layout，產生更易讀的網路圖 (使節點散開、避免節點重疊)
  net.repulsion()
  return net


In [20]:
net = matPresentGraph(mat = mat,node_id = pers)

net.show("./nissan_w2v.html")

291
291
291
./nissan_w2v.html


# Data Setting

In [48]:
metaData = pd.read_csv('./output/nissan_ptt_clean.csv')
metaData

,Unnamed: 0,system_id,artTitle,artDate,artPoster,artCatagory,word
0,0,1,[新聞]小休旅熱鬧好玩PEUGEOT30081.5LBlueH,2020/12/1 00:09,city0504,car,"['Corolla', 'Cross', '當紅', '炸子雞', 'NISSAN', 'K..."
1,0,1,[新聞]小休旅熱鬧好玩PEUGEOT30081.5LBlueH,2020/12/1 00:09,city0504,car,"['一網打盡', '不同', '消費', '需求', 'VW', 'TRocT', 'Cro..."
2,0,1,[新聞]小休旅熱鬧好玩PEUGEOT30081.5LBlueH,2020/12/1 00:09,city0504,car,"['尺碼', '規配', '迥異', '獨立', '車型', '吸引', '不同', '個性..."
3,0,1,[新聞]小休旅熱鬧好玩PEUGEOT30081.5LBlueH,2020/12/1 00:09,city0504,car,"['PEUGEOT', '3008', '單是', '引進', '國內', '車款', '選擇']"
4,0,1,[新聞]小休旅熱鬧好玩PEUGEOT30081.5LBlueH,2020/12/1 00:09,city0504,car,"['引擎動力', '汽油', '柴油']"
...,...,...,...,...,...,...,...
71827,2324,2325,[售車]自售NISSANTEANAJ3120052.0,2023/1/26 17:42,efs92e01,CarShop,"['電話', '0919911030LINE', '電話', '號碼', '聯繫']"
71828,2324,2325,[售車]自售NISSANTEANAJ3120052.0,2023/1/26 17:42,efs92e01,CarShop,"['不要', '半夜', '打給']"
71829,2325,2326,售Kicks原廠避光墊,2023/1/30 21:44,tingiy,CarShop,"['物品', '名稱', 'Nissan', 'kicks', '避光', '物品', '狀..."
71830,2325,2326,售Kicks原廠避光墊,2023/1/30 21:44,tingiy,CarShop,"['中壢', '車站', '輻射', '向外', '15', '公里', '宅配', '欲售..."


In [50]:
import seaborn as sns
from gensim.models.phrases import Phrases

Bigram


In [51]:
sents = metaData['word'].to_list()
bigrams = Phrases(sents, min_count=1, threshold=1000)
bigram_phrasers = Phrases(bigrams)
metaData['word_list_bigrams'] = list(bigram_phrasers[sents])

metaData.head()

TypeError: 'int' object is not iterable